# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import pprint as pp

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,280.00,96,100,2.19,CL,2024-10-19 03:30:50
1,1,kidal,18.4411,1.4078,309.91,11,11,2.28,ML,2024-10-19 03:30:51
2,2,grytviken,-54.2811,-36.5092,271.04,71,68,5.21,GS,2024-10-19 03:30:53
3,3,puerto baquerizo moreno,-0.9000,-89.6000,293.91,84,97,4.57,EC,2024-10-19 03:30:54
4,4,bodo,67.2800,14.4050,286.36,67,0,3.09,NO,2024-10-19 03:30:55


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    size='Humidity', 
    color='City',
    title='City Locations with Humidity',
    tiles='OSM',
    legend='bottom_right',
    alpha=0.6
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

pref_df = city_data_df[
    (city_data_df['Max Temp'] >= 73) 
    & (city_data_df['Wind Speed'] > 10) 
    & (city_data_df['Wind Speed'] < 15)
]

# Drop any rows with null values
# YOUR CODE HERE
pref_df = pref_df.dropna(axis=0)

# Display sample data
# YOUR CODE HERE
pref_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,nemuro,43.3236,145.5750,283.03,77,100,13.37,JP,2024-10-19 03:31:58
162,162,marsh harbour,26.5412,-77.0636,298.29,69,100,11.05,BS,2024-10-19 03:33:52
178,178,ribeira grande,38.5167,-28.7000,296.37,78,40,11.62,PT,2024-10-19 03:34:13
199,199,yuzhno-kurilsk,44.0306,145.8556,281.44,80,100,13.38,RU,2024-10-19 03:34:36
245,245,alvand,36.3187,49.1678,291.86,13,1,11.76,IR,2024-10-19 03:35:27
255,255,praia da vitoria,38.7333,-27.0667,296.29,73,20,14.17,PT,2024-10-19 03:35:38
280,280,shibetsu,43.6590,145.1320,281.01,81,100,10.55,JP,2024-10-19 03:36:08
396,396,sabang,5.8933,95.3214,301.03,83,100,11.77,ID,2024-10-19 03:38:22
493,493,dolinsk,47.3500,142.8000,274.87,88,45,11.39,RU,2024-10-19 03:40:19


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = pref_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = pd.Series(dtype='str')

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
61,nemuro,JP,43.3236,145.5750,77,NaN
162,marsh harbour,BS,26.5412,-77.0636,69,NaN
178,ribeira grande,PT,38.5167,-28.7000,78,NaN
199,yuzhno-kurilsk,RU,44.0306,145.8556,80,NaN
245,alvand,IR,36.3187,49.1678,13,NaN
255,praia da vitoria,PT,38.7333,-27.0667,73,NaN
280,shibetsu,JP,43.6590,145.1320,81,NaN
396,sabang,ID,5.8933,95.3214,83,NaN
493,dolinsk,RU,47.3500,142.8000,88,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'api_key': geoapify_key,
    'filter': '',
    'categories': 'accommodation',
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    res = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = res.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nemuro - nearest hotel: お宿エクハシ
marsh harbour - nearest hotel: D's Guest House
ribeira grande - nearest hotel: Quinta da Maria
yuzhno-kurilsk - nearest hotel: Айсберг
alvand - nearest hotel: هتل لاله هیدج
praia da vitoria - nearest hotel: Salles
shibetsu - nearest hotel: 楠旅館
sabang - nearest hotel: Hotel Montana
dolinsk - nearest hotel: Юбилейная


,City,Country,Lat,Lng,Humidity,Hotel Name
61,nemuro,JP,43.3236,145.5750,77,お宿エクハシ
162,marsh harbour,BS,26.5412,-77.0636,69,D's Guest House
178,ribeira grande,PT,38.5167,-28.7000,78,Quinta da Maria
199,yuzhno-kurilsk,RU,44.0306,145.8556,80,Айсберг
245,alvand,IR,36.3187,49.1678,13,هتل لاله هیدج
255,praia da vitoria,PT,38.7333,-27.0667,73,Salles
280,shibetsu,JP,43.6590,145.1320,81,楠旅館
396,sabang,ID,5.8933,95.3214,83,Hotel Montana
493,dolinsk,RU,47.3500,142.8000,88,Юбилейная


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hotel_df['hover'] = hotel_df.apply(
    lambda row: f"{row['City']}: {row['Hotel Name']} ({row['Country']})", axis=1
)

# Configure the map plot
# YOUR CODE HERE
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat', 
    size='Humidity', 
    color='City',
    hover_cols=['hover'],
    title='City Hotel Locations',
    tiles='OSM',
    legend='bottom_right',
    alpha=0.6
)

# Display the map
# YOUR CODE HERE
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Humidity,hover)